In [1]:
# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import getpass
import pdvega
import plotly.graph_objs as go

from plotly.offline import iplot, init_notebook_mode
import plotly.io as pio
from plotly.graph_objs import *

# for configuring connection 
from configobj import ConfigObj
import os

%matplotlib inline


import os


from sklearn import linear_model
from sklearn import metrics
from sklearn.model_selection import train_test_split

#configure the notebook for use in offline mode
init_notebook_mode(connected=True)

In [2]:
df= pd.read_csv("analysis.csv")
df.head()

,Unnamed: 0,hospitalid,sodium,electivesurgery,vent,dialysis,gcs,urine,wbc,temperature,...,m11_True,m12_True,m13_True,m14_True,m15_True,m16_True,m17_True,m18_True,m19_True,m20_True
0,0,59.0,139.0,-1.0,0.0,0.0,15.0,-1.0,14.7,36.1,...,1,0,0,1,1,0,0,0,1,0
1,1,73.0,134.0,-1.0,0.0,0.0,13.0,-1.0,14.1,39.3,...,1,0,0,1,1,0,0,0,1,0
2,2,73.0,-1.0,1.0,1.0,0.0,15.0,-1.0,8.0,34.8,...,0,0,1,0,0,1,0,1,0,0
3,3,63.0,137.0,-1.0,0.0,0.0,15.0,-1.0,10.9,36.6,...,1,0,1,1,1,0,0,1,1,0
4,4,63.0,135.0,-1.0,0.0,0.0,15.0,-1.0,5.9,35.0,...,0,0,1,0,0,0,0,1,0,0


In [3]:
df.shape

(95148, 85)

In [4]:
del df['hospitalid']

df = df.drop(df.columns[[0]], axis=1)
df = df.drop(df.columns[[63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82]], axis=1)

In [5]:
missing_values_count = df.isnull().sum()
#df.replace('-1.0', np.nan)
df = df.replace({-1.0:np.nan, -1.0:np.nan})
df.head()
missing_values_count = df.isnull().sum()
missing_values_count

sodium                        18244
electivesurgery               74997
vent                              0
dialysis                          0
gcs                            1728
urine                         45829
wbc                           22141
temperature                    4139
respiratoryrate                 582
heartrate                       188
meanbp                          263
creatinine                    18332
ph                            73474
hematocrit                    20021
albumin                       58143
pao2                          73474
pco2                          73474
bun                           18774
glucose                       10909
bilirubin                     60797
fio2                          73474
age                            3356
thrombolytics                     0
aids                              0
hepaticfailure                    0
lymphoma                          0
metastaticcancer                  0
leukemia                    

**We moved all the pre-processing including splitting>imputation>Standardization to the CV iterations**

In [6]:
cols_to_norm=['gcs', 'urine', 'wbc', 'sodium',
       'temperature', 'respiratoryrate', 'heartrate', 'meanbp', 'creatinine',
       'ph', 'hematocrit', 'albumin', 'pao2', 'pco2', 'bun', 'glucose',
       'bilirubin', 'fio2', 'age', 'offset']

X=df.drop('destcopy', 1)
y=df['destcopy']
df_cols = list(X)     #fancy impute removes column names.

In [7]:
df.columns

Index(['sodium', 'electivesurgery', 'vent', 'dialysis', 'gcs', 'urine', 'wbc',
       'temperature', 'respiratoryrate', 'heartrate', 'meanbp', 'creatinine',
       'ph', 'hematocrit', 'albumin', 'pao2', 'pco2', 'bun', 'glucose',
       'bilirubin', 'fio2', 'age', 'thrombolytics', 'aids', 'hepaticfailure',
       'lymphoma', 'metastaticcancer', 'leukemia', 'immunosuppression',
       'cirrhosis', 'readmit', 'offset', 'destcopy', 'admitsource_1.0',
       'admitsource_2.0', 'admitsource_3.0', 'admitsource_4.0',
       'admitsource_5.0', 'admitsource_6.0', 'admitsource_7.0',
       'admitsource_8.0', 'diaggroup_ARF', 'diaggroup_Asthma-Emphys',
       'diaggroup_CABG', 'diaggroup_CHF', 'diaggroup_CVA', 'diaggroup_CVOther',
       'diaggroup_CardiacArrest', 'diaggroup_ChestPainUnknown',
       'diaggroup_Coma', 'diaggroup_DKA', 'diaggroup_GIBleed',
       'diaggroup_GIObstruction', 'diaggroup_Neuro', 'diaggroup_Other',
       'diaggroup_Overdose', 'diaggroup_PNA', 'diaggroup_RespMedOther',


**XGB**

In [ ]:
%matplotlib inline
from sklearn.model_selection import KFold
from sklearn import preprocessing
from imblearn.over_sampling import SMOTE
from sklearn.metrics import f1_score
from imblearn.metrics import classification_report_imbalanced
# explicitly require this experimental feature

from yellowbrick.classifier import ROCAUC
from sklearn.linear_model import LogisticRegression
from numpy import loadtxt
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import io 


classes=['Death','Home','Nursing Home','Rehabilitation']


kf = KFold(n_splits=10)


for i in range (1,11):

    for fold, (train_index, test_index) in enumerate(kf.split(X), 1):
        X_train = X.iloc[train_index]
        y_train = y.iloc[train_index]  # Based on your code, you might need a ravel call here, but I would look into how you're generating your y
        X_test = X.iloc[test_index]
        y_test = y.iloc[test_index]  # See comment on ravel and  y_train


    #------------------------------IMPUTE Training Set------------------------------------

    

    #------------------------------Standardize Testing Set------------------------------------

        std_scale = preprocessing.StandardScaler().fit(X_train[cols_to_norm])
        X_train[cols_to_norm] = std_scale.transform(X_train[cols_to_norm])
        X_test[cols_to_norm] = std_scale.transform(X_test[cols_to_norm])
    #------------------------------------------------------------------------------------------

     # Hyperparameters are optimized using hyperopt

        #sm = SMOTE()
        #X_train_oversampled, y_train_oversampled = sm.fit_sample(X_train, y_train)
        model = XGBClassifier(max_depth=8, gamma=0.063, colsample_bytree=0.71)
        model.fit(X_train, y_train)  
        y_pred = model.predict(X_test)
        visualizer = ROCAUC(model, classes=classes)
        visualizer.fit(X_train, y_train)  # Fit the training data to the visualizer
        visualizer.score(X_test, y_test)  # Evaluate the model on the test data
        visualizer.poof("XGB_Unbalanced_{}_{}.pdf".format(i, fold), clear_figure=True) 
        print(f'For fold {fold}:')
        print(f'Accuracy: {model.score(X_test, y_test)}')
        f1=f1_score(y_test, y_pred, average='micro')
        print(f'f-score: {f1}')
        print(classification_report_imbalanced(y_test, y_pred))
        K= classification_report_imbalanced(y_test, y_pred)
        df = pd.read_fwf(io.StringIO(K))
        df.loc["1":"1","pre":"sup"].to_csv("XGB-U-D.csv" , sep=',', encoding='utf-8', doublequote=False, index=False, mode="a", header=False)
        df.loc["2":"2","pre":"sup"].to_csv("XGB-U-H.csv" , sep=',', encoding='utf-8', doublequote=False, index=False, mode="a", header=False)
        df.loc["3":"3","pre":"sup"].to_csv("XGB-U-N.csv" , sep=',', encoding='utf-8', doublequote=False, index=False, mode="a", header=False)
        df.loc["4":"4","pre":"sup"].to_csv("XGB-U-R.csv" , sep=',', encoding='utf-8', doublequote=False, index=False, mode="a", header=False)
        df.iloc[6:7,:].to_csv("XGB-U-avg.csv" , sep=',', encoding='utf-8', doublequote=False, index=False, mode="a", header=False)
        from sklearn.metrics import confusion_matrix
        print(confusion_matrix(y_test, y_pred))

        #




/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:42: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:43: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-v

For fold 1:
Accuracy: 0.7585916973200211
f-score: 0.7585916973200211
                   pre       rec       spe        f1       geo       iba       sup

          1       0.54      0.48      0.97      0.51      0.68      0.45       642
          2       0.79      0.96      0.37      0.87      0.59      0.37      6776
          3       0.58      0.24      0.96      0.34      0.48      0.21      1716
          4       0.31      0.01      1.00      0.02      0.10      0.01       381

avg / total       0.72      0.76      0.54      0.71      0.56      0.33      9515

[[ 310  239   89    4]
 [ 102 6493  176    5]
 [ 137 1168  411    0]
 [  21  325   31    4]]


/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:42: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:43: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-v

For fold 2:
Accuracy: 0.7777193904361535
f-score: 0.7777193904361535
                   pre       rec       spe        f1       geo       iba       sup

          1       0.60      0.49      0.96      0.54      0.69      0.45       974
          2       0.87      0.90      0.48      0.88      0.66      0.45      7520
          3       0.20      0.27      0.92      0.23      0.49      0.23       697
          4       0.09      0.01      1.00      0.01      0.08      0.01       324

avg / total       0.76      0.78      0.58      0.77      0.63      0.42      9515

[[ 477  347  149    1]
 [ 199 6735  567   19]
 [  99  412  186    0]
 [  17  272   33    2]]


/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:42: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:43: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-v

...

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:42: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:43: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-v

For fold 4:
Accuracy: 0.7147661586967945
f-score: 0.7147661586967945
                   pre       rec       spe        f1       geo       iba       sup

          1       0.67      0.51      0.97      0.58      0.70      0.47      1129
          2       0.74      0.95      0.35      0.83      0.57      0.35      6348
          3       0.35      0.15      0.96      0.21      0.38      0.14      1285
          4       0.36      0.01      1.00      0.01      0.07      0.00       753

avg / total       0.65      0.71      0.56      0.66      0.52      0.31      9515

[[ 573  433  122    1]
 [ 118 6026  202    2]
 [ 124  959  198    4]
 [  38  672   39    4]]


/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:42: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:43: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-v

For fold 5:
Accuracy: 0.7389385181292696
f-score: 0.7389385181292696
                   pre       rec       spe        f1       geo       iba       sup

          1       0.64      0.50      0.96      0.56      0.69      0.46      1085
          2       0.77      0.95      0.41      0.85      0.63      0.41      6437
          3       0.50      0.21      0.96      0.29      0.45      0.18      1657
          4       0.14      0.01      1.00      0.02      0.09      0.01       336

avg / total       0.69      0.74      0.59      0.69      0.58      0.36      9515

[[ 538  391  148    8]
 [ 123 6145  164    5]
 [ 154 1152  345    6]
 [  26  272   35    3]]


/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:42: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:43: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-v